In [1]:
import pandas as pd
from transformers import pipeline
from datasets import load_dataset
from tqdm.auto import tqdm
import torch

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-11 04:40:01.605267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744346401.775756    4609 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744346401.826345    4609 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-11 04:40:02.232264: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical opera

In [2]:
import pandas as pd
df = pd.read_parquet('../data/5b.cryptonews_topic_sentiment.parquet')
df.head()

,title,text,source_name,date,topic,topic_confidence_score,news_text_title,sentiment_finbert,score_finbert,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa,People,Organizations,Cryptocurrencies,Events
0,2021 Bitcoin Price Predictions: Is The Massive...,As the bitcoin price hovers under the psycholo...,Forbes,2021-01-02 00:20:00+00:00,"liquidity, volatility, price action, price mov...",0.238574,2021 Bitcoin Price Predictions: Is The Massive...,neutral,0.876394,0.5,0.0,0.0,0.0,0.0,0.0,[],[],[Bitcoin],[]
1,Will Central Banks Hold Bitcoin in 2021?,Central banks (CB) will hold bitcoin sooner or...,BeInCrypto,2021-01-01 20:31:35+00:00,consensus updates,0.199005,Will Central Banks Hold Bitcoin in 2021? Centr...,neutral,0.916949,0.0,0.5,0.0,0.0,0.7,0.0,[Krüger],[Central banks],[Bitcoin],[]
2,"Bitcoin Is Digital Social Justice, feat. Tyron...",The podcaster and CEO of Onramp Invest discuss...,Coindesk,2021-01-01 19:15:02+00:00,social platforms,0.299585,"Bitcoin Is Digital Social Justice, feat. Tyron...",neutral,0.903567,0.0,0.3,0.0,0.5,0.7,0.0,[Tyrone Ross],[Onramp Invest],[Bitcoin],[]
3,Bitcoin hits all-time high against gold as hav...,"BTC has hit another milestone, this time again...",Cointelegraph,2021-01-01 18:52:00+00:00,"liquidity, volatility, price action, price mov...",0.430232,Bitcoin hits all-time high against gold as hav...,positive,0.931128,1.0,0.0,0.0,0.0,0.0,0.0,[],[],"[BTC, Bitcoin]",[]
4,"The Last Time This Indicator Flashed, Bitcoin ...",Bitcoin has been facing some turbulence as of ...,Bitcoinist,2021-01-01 18:00:00+00:00,"liquidity, volatility, price action, price mov...",0.522232,"The Last Time This Indicator Flashed, Bitcoin ...",negative,0.716469,0.3,0.0,0.0,0.0,0.0,0.0,[],[],"[Bitcoin, BTC]",[]


### Approach 1: pretrained model - Jean-Baptiste/roberta-large-ner-english
- poor performance

In [3]:
dataset = load_dataset('parquet', data_files='../data/5b.cryptonews_topic_sentiment.parquet')['train']
print(dataset)

# select first 1000 rows
dataset = dataset.select(range(1000))

dataset = dataset.map(lambda example: {
  'news_text_title': example['title'] + ' ' + example['text']
})

Generating train split: 155376 examples [00:00, 486035.71 examples/s]


Dataset({
    features: ['title', 'text', 'source_name', 'date', 'topic', 'topic_confidence_score', 'news_text_title', 'sentiment_finbert', 'score_finbert', 'tanalysis_absa', 'economy_absa', 'regulation_absa', 'technology_absa', 'adoption_absa', 'cybersecurity_absa', 'People', 'Organizations', 'Cryptocurrencies', 'Events'],
    num_rows: 155376
})


Map: 100%|██████████| 1000/1000 [00:00<00:00, 5442.25 examples/s]


In [4]:
# test
# NER model from Hugging Face
pipe = pipeline("token-classification", model="Jean-Baptiste/roberta-large-ner-english")
txt = "Bank of America: Bitcoin Bull Run Is the 'Mother of All Bubbles' A senior analyst at Bank of America thinks that Bitcoin's latest bull run “blows the doors off prior bubbles."
ner_result = pipe(txt)

RuntimeError: Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# NER model from Hugging Face
pipe = pipeline("token-classification", model="Jean-Baptiste/roberta-large-ner-english")

def NER(batch):
  batch['NER'] = pipe(batch['news_text_title'])
  return batch

dataset = dataset.map(NER, batched=True, batch_size=5)

Device set to use cuda:0


In [ ]:
def post_process_ner(ner_output):
  processed_entities = []
  current_entity = {}
  
  for token in ner_output:
    if current_entity and token['entity'] == current_entity['entity'] and token['start'] == current_entity['end'] + 1:
      current_entity['word'] += token['word'].replace('Ġ', ' ')
      current_entity['end'] = token['end']
    else:
      if current_entity:
        processed_entities.append(current_entity)
      current_entity = token.copy()
  
  if current_entity:
    processed_entities.append(current_entity)
  
  return processed_entities

def get_entities(processed_ner_output, entity_type=None):
  if entity_type is None:
    return {entity['word'][1:] for entity in processed_ner_output}
  else:
    return {entity['word'][1:] for entity in processed_ner_output if entity['entity'] == entity_type}


# aply post-processing and get entities from NER results
dataset = dataset.map(lambda batch: {
  'entities': get_entities(post_process_ner(batch['NER']))
})

In [ ]:
df = dataset.to_pandas()
df[['title','text','entities']]

# poor performance, need to look for another NER approach

,title,text,entities
0,2021 Bitcoin Price Predictions: Is The Massive...,As the bitcoin price hovers under the psycholo...,[]
1,Will Central Banks Hold Bitcoin in 2021?,Central banks (CB) will hold bitcoin sooner or...,"[Kr, er, ¼]"
2,"Bitcoin Is Digital Social Justice, feat. Tyron...",The podcaster and CEO of Onramp Invest discuss...,"[, Tyr, i, On, ne Ross, mp Invest, De]"
3,Bitcoin hits all-time high against gold as hav...,"BTC has hit another milestone, this time again...",[BTC]
4,"The Last Time This Indicator Flashed, Bitcoin ...",Bitcoin has been facing some turbulence as of ...,"[BTC, Bitcoin]"
...,...,...,...
995,Associated Gas Pilot Could Lead to Huge Boon f...,Russian Bitcoin (BTC) miners could be in for a...,"[Russian, , ns, t, TC, Gaz, Kh, rom, Russian B..."
996,"Bitcoin Mining Revenue Hits $20 Billion, Doubl...",Bitcoin miners are making a killing from this ...,[]
997,Low-Fee Bitcoin Trust to Challenge Grayscale,Custodial Fee Wars Cryptocurrency is still bar...,"[, Bit, IT, B, ise 10 Crypto Index Fund, ys, a..."
998,Bitcoin Faced First Major Correction In Curren...,This was one of the most intense weeks. Bitcoi...,"[Update, The Crypto Weekly]"


### Approach 2: fine-tune a model


In [ ]:
'''
Choose a Base Model: bert-base-uncased, roberta-base, ElKulako/cryptobert, ProsusAI/finbert, or modernbert?
Load NER dataset - ???????
Tokenize and Align Labels
Fine-tune the Model
Evaluate the Model (metrics like precision, recall, and F1-score)

Problem: Do we have suitable crypto news NER dataset?
'''

'\nChoose a Base Model\nGeneral-purpose: bert-base-uncased, roberta-base\nDomain-specific: ElKulako/cryptobert (for crypto), ProsusAI/finbert (finance), or modernbert?\n\nLoad NER dataset (e.g., CoNLL-2003, or any other crypo news dataset with NER annotations)\nTokenize and Align Labels\nFine-tune the Model\nEvaluate the Model (metrics like precision, recall, and F1-score)\n'